In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import sys

- asos.com (user-agent,no igredients)
- zalando (user-agent)
- sephora 
- avon.com (kz- no ingredients, com has)
- monamie.kz (no igredients)
- feelunique.com [интересный сайт косметики](https://row.feelunique.com/p/Charlotte-Tilbury-Charlottes-Magic-Cream-50ml)
- паряться насчет состава [insta page](https://www.instagram.com/pure_beautystore/?igshid=ev5m6oyq06zy)

In [2]:
home_url = 'https://en.zalando.de'

In [3]:
#categories = ['beauty-complexion', 'beauty-skincare-face','beauty-sun-protection']
category = 'beauty-skincare'

In [4]:
page_number = 1

https://en.zalando.de/beauty-skincare/?p=2

In [5]:
https_line = home_url +'/' + str(category) + '/?p=' + str(page_number)

In [6]:
https_line

'https://en.zalando.de/beauty-skincare/?p=1'

- надо браузерские headers дать, как-будто ты user-agent. 
[тут дают решение](https://stackoverflow.com/questions/47375153/requests-get-in-python-giving-connection-timeout-error)

In [7]:
headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36'}

In [8]:
response = requests.get(https_line, headers=headers)
soup = BeautifulSoup(response.content,'lxml')

In [9]:
obj = soup.find('a', attrs = {'class':'cat_imageLink-OPGGa'})
obj.attrs['href']

'/darphin-intral-serum-30ml-serum-dao31g008-s11.html'

In [10]:
product_links = soup.findAll('a', attrs = {'class':'cat_imageLink-OPGGa'})
product_links = [link.attrs['href'] for link in product_links]

In [12]:
home_url

'https://en.zalando.de'

In [13]:
def getPageLinks(category, page_number):
      
    beauty_link = home_url +'/'+ str(category) + '/?p=' + str(page_number)
  
    response = requests.get(beauty_link, headers=headers)
    
    if not response.ok:
        return response.status_code
    
    html = response.content
    soup = BeautifulSoup(html,'lxml')
    
    product_links = soup.findAll('a', attrs = {'class':'cat_imageLink-OPGGa'})
    product_links = [ home_url + link.attrs['href'] for link in product_links]
    
    return product_links

In [14]:
links = getPageLinks(category,2)
links[:3]

['https://en.zalando.de/clinique-sos-kit-back-to-school-fresh-as-a-freshman-skincare-set-cll31g055-s11.html',
 'https://en.zalando.de/anatomicals-salt-scrub-650g-body-exfoliator-neutral-an434g008-s12.html',
 'https://en.zalando.de/clinique-moisture-surge-hydrating-supercharged-concentrate-48ml-skin-moisturizer-neutral-cll31g015-s11.html']

In [15]:
links = getPageLinks(category,1)
links[:3]

['https://en.zalando.de/origins-ginzingtm-ultra-hydrating-energy-boosting-cream-upgrade-50ml-face-cream-orb34g002-s11.html',
 'https://en.zalando.de/rosental-organics-rose-queen-roller-skincare-tool-rosenquarz-rol31j001-j11.html',
 'https://en.zalando.de/clinique-moisture-surge-72-hour-auto-replenishing-hydrator-30ml-skin-moisturizer-neutral-cll31g011-s11.html']

In [17]:
#category
# df = pd.DataFrame(columns=['category'])
df1 = pd.DataFrame(columns=['brand', 'name', 'price', 'rating', 'skin_type', 'ingredients'])
# df = pd.concat([df, df1], axis = 1)

In [18]:
df1

,brand,name,price,rating,skin_type,ingredients


In [19]:
# testing on first link
product_page = 'https://en.zalando.de/pixi-glow-tonic-250ml-toner-tonic-pik31g000-s11.html'

time.sleep(15)
try:
    response = requests.get(product_page, headers=headers)
    html = response.content
    soup = BeautifulSoup(html,'lxml')
except requests.exceptions.ConnectionError:
    status_code = "Connection refused. Too many requests from this ip"

id = z-vegas-pdp-props

#### Brand name

In [20]:
soup.find('span', attrs={'class':'A95iT1 pDVUjz nmA88J NNECXo AHAcbe x--xNS'}).text

'Pixi'

In [21]:
def get_brand(soup):
    brand = soup.find('span', attrs={'class':'A95iT1 pDVUjz nmA88J NNECXo AHAcbe x--xNS'})
    return "" if not brand else brand.text

In [22]:
new_product_page = links[3]
new_response = requests.get(new_product_page, headers=headers)
new_html = new_response.content
new_soup = BeautifulSoup(new_html,'lxml')

In [23]:
new_product_page

'https://en.zalando.de/rosental-organics-jade-roller-jade-princess-skincare-tool-rol31j000-m11.html'

In [24]:
get_brand(new_soup)

'Rosental Organics'

#### Product name

In [25]:
soup.find('h1', attrs = {'class':'A95iT1 pDVUjz nmA88J FI5Jsp x--xNS Y6OuCD BicgmA'}).text

'GLOW TONIC 250ML - Toner'

In [26]:
def get_name(soup):
    name = soup.find('h1', attrs = {'class':'A95iT1 pDVUjz nmA88J FI5Jsp x--xNS Y6OuCD BicgmA'})
    return "" if not name else name.text

In [27]:
get_name(new_soup)

'ROLLER - Skincare tool'

#### Product price

In [28]:
text = soup.find('span', attrs = {'class':'A95iT1 pDVUjz nmA88J _0uQAcH AHAcbe A_8GRa Xb35xC'}).text
text

'23,95\xa0€'

In [29]:
pattern = re.compile(r'\d+\,?\d*')
result = re.match(pattern, text)

In [30]:
price = result.group(0)

In [31]:
float(price.replace(',','.'))

23.95

In [32]:
def get_price(soup):
    price_tag = soup.find('span', attrs = {'class':'A95iT1 pDVUjz nmA88J _0uQAcH AHAcbe A_8GRa Xb35xC'})
    if price_tag:
        price_phrase = price_tag.text
    else:
        discount = soup.find('span', attrs={'class':'A95iT1 pDVUjz nmA88J _0uQAcH AHAcbe gzB009'})
        price_phrase = discount.text
        
    pattern = re.compile(r'\d+\,?\d*')
    result = re.match(pattern, price_phrase)
    price = result.group(0)
    return float(price.replace(',','.'))

In [33]:
get_price(new_soup)

29.95

In [34]:
get_price(soup)

23.95

#### Skin type

In [35]:
product_details = soup.find('div', attrs = {'role':'tabpanel'})

In [36]:
tab_contents = product_details.findAll('div', attrs = {'class':'h-tabs__panel'})

In [37]:
highlights = tab_contents[0]

In [38]:
highlights_text = highlights.findAll('span', attrs = {'class':'body', 'as':'span'})

In [41]:
highlights_dict = {}

In [42]:
for i in range(0,len(highlights_text),2):
    clear_text = highlights_text[i].text.split(':')[0]
    highlights_dict[clear_text] = highlights_text[i+1].text

In [43]:
highlights_dict

{'Article number': 'PIK31G000-S11',
 'Concerns': 'Dryness, Dullness, Uneven Texture, Oiliness, Uneven skin tone, Acne',
 'Consistency': 'Liquid',
 'Skin type': 'All skin types',
 'Volume': '250 ml',
 'Warnings': 'We recommend using SPF in the daytime, especially when using a product with glycolic acid.'}

In [44]:
def get_skin_type(soup):
    product_details = soup.find('div', attrs = {'role':'tabpanel'})
    tab_contents = product_details.findAll('div', attrs = {'class':'h-tabs__panel'})
    highlights = tab_contents[0]
    highlights_text = highlights.findAll('span', attrs = {'class':'body', 'as':'span'})
    highlights_dict = {}
    for i in range(0,len(highlights_text),2):
        clear_text = highlights_text[i].text.split(':')[0]
        highlights_dict[clear_text] = highlights_text[i+1].text
        
    return "" if 'Skin type' not in highlights_dict else highlights_dict['Skin type']

In [45]:
get_skin_type(soup)

'All skin types'

In [46]:
page = 'https://en.zalando.de/pixi-glow-tonic-250ml-toner-tonic-pik31g000-s11.html'
response2 = requests.get(page, headers=headers)
html2= response2.content
soup2 = BeautifulSoup(html2,'lxml')

In [47]:
get_skin_type(new_soup)

''

#### Ingredients

In [96]:
ingredients = tab_contents[1]

In [97]:
titles = ingredients.findAll('span', attrs = {'class':'h-text h-color-black body h-bold'})

In [98]:
titles[0].text

'Ingredients:\xa0'

In [99]:
ingredients_pos = -1 

In [100]:
for i in range(0, len(titles)):
    if 'Ingredients:' in titles[i].text:
        ingredients_pos = i 

In [101]:
ingredients_pos

0

In [111]:
contents = ingredients.find('p', attrs = {'class':'h-text h-color-black body h-m-bottom-xs'})

In [112]:
contents

<p as="p" class="h-text h-color-black body h-m-bottom-xs"><span as="span" class="h-text h-color-black body h-bold">Ingredients<!-- -->: </span><span as="span" class="h-text h-color-black body"><div class="h-container h-align-left"><p as="p" class="h-text h-color-black body">Aqua, Aloe Barbadensis Leaf Juice, Hamamelis Virginiana (Witch Hazel) Leaf Extract, Aesculus Hippocastanum (Horse Chestnut) Seed Extract, Glycolic Acid, Ammonium Glycolate, Glycerin, Butylene Glycol, Hexylene Glycol (and) Fructose (and) Glucose (a...</p><button as="button" class="h-action h-container--slide__link--black h-m-top-s -link underline"><span class="h-text h-color-black body -text-only"><h5 class="h-text h-color-black title-4">Discover more</h5></span></button></div></span></p>

In [113]:
contents.find('p', attrs = {'class':'h-text h-color-black body'}).text

'Aqua, Aloe Barbadensis Leaf Juice, Hamamelis Virginiana (Witch Hazel) Leaf Extract, Aesculus Hippocastanum (Horse Chestnut) Seed Extract, Glycolic Acid, Ammonium Glycolate, Glycerin, Butylene Glycol, Hexylene Glycol (and) Fructose (and) Glucose (a...'

In [56]:
contents[ingredients_pos].text

'Aqua, Aloe Barbadensis Leaf Juice, Hamamelis Virginiana (Witch Hazel) Leaf Extract, Aesculus Hippocastanum (Horse Chestnut) Seed Extract, Glycolic Acid, Ammonium Glycolate, Glycerin, Butylene Glycol, Hexylene Glycol (and) Fructose (and) Glucose (a...'

In [57]:
"z-vegas-details-props"

'z-vegas-details-props'

In [114]:
def get_ingredients(soup):    
    product_details = soup.find('div', attrs = {'role':'tabpanel'})
    tab_contents = product_details.findAll('div', attrs = {'class':'h-tabs__panel'})
    ingredients = tab_contents[1]
    titles = ingredients.findAll('span', attrs = {'class':'h-text h-color-black body h-bold'})
    contents = ingredients.findAll('p', attrs = {'class':'h-text h-color-black body h-m-bottom-xs'})
    ingredients_pos = -1
    for i in range(0, len(titles)):
        if 'Ingredients:' in titles[i].text:
            ingredients_pos = i 
    return "" if ingredients_pos == -1 else contents[0].find('p', attrs = {'class':'h-text h-color-black body'}).text

In [115]:
get_ingredients(new_soup)

''

In [116]:
get_ingredients(soup2)

'Aqua, Aloe Barbadensis Leaf Juice, Hamamelis Virginiana (Witch Hazel) Leaf Extract, Aesculus Hippocastanum (Horse Chestnut) Seed Extract, Glycolic Acid, Ammonium Glycolate, Glycerin, Butylene Glycol, Hexylene Glycol (and) Fructose (and) Glucose (a...'

#### Rating

In [63]:
def get_average_rating(html_content):
    search_token = 'averageStarRating'
    offset = len(search_token) + 2
    start_pos = html_content.find(search_token)
    colon_pos = html_content.find(":", start_pos)
    comma_pos = html_content.find(",", start_pos)
    start_pos, colon_pos, comma_pos
    rating = html_content[colon_pos+1: comma_pos]
    return "" if rating=='null' else float(rating)

In [64]:
for l in links[:3]:
    html_content = str(requests.get(l, headers=headers).content)
    print(get_average_rating(html_content))
    time.sleep(1)

4.4615383
4.59542
4.707317


In [65]:
li = 'https://en.zalando.de/clinique-superdefense-cream-spf-25-skin-type-34-face-cream-cll31g04z-s11.html'
html_content = str(requests.get(li, headers=headers).content)
get_average_rating(html_content)


''

In [67]:

def get_product_data(product_page):
    """
        returns dictionary data_row
        
        product_page: string
            url for the product html page
    
    """
    
    response = requests.get(product_page, headers=headers)
    
    if not response.ok:
        return response.status_code
    
    html = response.content
    soup = BeautifulSoup(html,'lxml')
    html_content = str(response.content)
    
    product_brand = get_brand(soup)
    product_name = get_name(soup)
    product_price = get_price(soup)
    
    product_rating = get_average_rating(html_content)
    ingredients = get_ingredients(soup)
    skin_type = get_skin_type(soup)


    data_row = {"brand":product_brand,
                "name":product_name, 
                "price":product_price, 
                "rating":product_rating,
                "skin_type":skin_type, 
                "ingredients":ingredients
               }

    return data_row

In [68]:
new_product_page = links[3]
new_product_page

'https://en.zalando.de/rosental-organics-jade-roller-jade-princess-skincare-tool-rol31j000-m11.html'

In [69]:
data_row = get_product_data(new_product_page)

In [70]:
df1 = df1.append(data_row, ignore_index=True)

In [71]:
df1

,brand,name,price,rating,skin_type,ingredients
0,Rosental Organics,ROLLER - Skincare tool,29.95,4.658878,,


In [79]:
links[11]

'https://en.zalando.de/egyptian-magic-egyptian-magic-skin-cream-59ml-skin-moisturizer-neutral-eg234g002-s11.html'

In [117]:
%%time
for product_link in links:
    data_row = get_product_data(product_link)
    df1 = df1.append(data_row, ignore_index=True)

CPU times: user 3.4 s, sys: 93.4 ms, total: 3.49 s
Wall time: 17.9 s


In [118]:
df1

,brand,name,price,rating,skin_type,ingredients
0,Origins,GINZING™ ULTRA-HYDRATING ENERGY-BOOSTING CREAM...,15.95,4.46154,All skin types,"Water\Aqua\Eau, Caprylic/Capric Triglyceride, ..."
1,Rosental Organics,ROSE QUEEN ROLLER - Skincare tool,34.95,4.59542,,
2,Clinique,MOISTURE SURGE 72-HOUR AUTO-REPLENISHING HYDRA...,19.00,4.70732,Dry skin,"Water\Aqua\Eau , Dimethicone , Butylene Glycol..."
3,Rosental Organics,ROLLER - Skincare tool,29.95,4.65888,,
4,Pai skincare,ROSEHIP BIOREGENERATE OIL MINI - Face oil,15.95,4.5,All skin types,99% ORGANIC Rosa Canina Seed Extract* - Rosehi...
5,Pixi,GLOW TONIC 250ML - Toner,23.95,4.7438,All skin types,"Aqua, Aloe Barbadensis Leaf Juice, Hamamelis V..."
6,Mimitika,SPF 50 FACE SUNSCREEN 50ML - Sun protection,15.95,4.6875,All skin types,AQUA (WATER) – CYCLOPENTASILOXANE – ETHYLHEXYL...
7,Pixi,GLOW TONIC 100ML - Toner,11.95,4.65865,All skin types,"Aqua, Aloe Barbadensis Leaf Juice, Hamamelis V..."
8,Clinique,SUPERDEFENSE CREAM SPF 25 SKIN TYPE 3/4 - Face...,33.95,,All skin types,"WATER\AQUA\EAU, ETHYLHEXYL SALICYLATE, GLYCERI..."
9,Sand&Sky,AUSTRALIAN PINK CLAY POREFINING FACE MASK 60G ...,49.95,4.52941,All skin types,"Kaolin (Australian Clay), Aqua (Water), Benton..."


In [119]:
df1.to_csv('data/draft.csv', encoding = 'utf-8', index = False)